In [49]:
from __future__ import print_function, division
from PIL import Image
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import natsort
from spectral import *
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import utils, transforms 
from torch.utils.data import DataLoader
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [195]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, image_dir, no_background_dir, no_background_imgdir, transform, is_resize, img_list):
        self.main_dir = main_dir
        self.image_dir = image_dir
        self.no_background_dir = no_background_dir
        self.no_background_imgdir = no_background_imgdir
        self.transform = transform
        self.is_resize = is_resize
        self.img_list = img_list
        #all_imgs = os.listdir(main_dir)
        self.total_imgs = img_list['img']    #natsort.natsorted(img_list['img'])

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        if self.is_resize:
            hdr_loc = os.path.join(self.no_background_dir, self.total_imgs[idx].replace("resized", "no_note"))
            img_loc = os.path.join(self.no_background_imgdir, self.total_imgs[idx].replace("resized.hdr", "no_note.img"))
            image = envi.open(hdr_loc, img_loc)
            resized = resize(image, size)
            """get mean and std from top 2 rows for background"""
            mean = np.median(image[:, 0:1, :], axis=(0, 1))  # median for every band
            black_std = np.std(image[:, 0:1, :], axis=(0, 1))
            final_image = pad_img(resized, size, mean, black_std, img)
        else:
            hdr_loc = os.path.join(self.main_dir, self.total_imgs[idx])
            img_loc = os.path.join(self.image_dir, self.total_imgs[idx].replace(".hdr", ".img"))
            image = envi.open(hdr_loc, img_loc)
            final_image = np.zeros((image.shape[0], image.shape[1], image.shape[2]))
            final_image[:, :, :] = image[:, :, :]
        x=[]
        #set a seed so the same transforms are applied to each channel
        seed = np.random.randint(2147483647)
        for ch in range(final_image.shape[2]):
            random.seed(seed)
            x.append(transform(Image.fromarray(final_image[:, :, ch])))
        #this is the multichannel transformed image (a torch tensor)
        img_tfm = torch.cat(x)
        y = self.img_list['label'][idx].values
        print(y)
        return img_tfm, y

storage_path = "D:/My Drive/StoragePath"
result_path = storage_path + "/ExpResults/tal_exp_results"
resized_path = result_path + "/resized"
resized_img_path = result_path + "/resizes_img"
no_note_path = result_path + "/no_note_background"
no_note_img_path = result_path + "/no_note_background_img"

tfms = transforms.Compose([transforms.RandomHorizontalFlip(),
                   transforms.ToTensor()])

train_dataset = CustomDataSet(resized_path, resized_img_path, no_note_path, no_note_img_path, transform=tfms, is_resize=False, img_list = df_train)
val_dataset = CustomDataSet(resized_path, resized_img_path, no_note_path, no_note_img_path, transform=tfms, is_resize=False, img_list = df_val)
test_dataset = CustomDataSet(resized_path, resized_img_path, no_note_path, no_note_img_path, transform=tfms, is_resize=False, img_list = df_test)
# x, y = random_split(my_dataset, [20, 9])

train_loader = DataLoader(train_dataset , batch_size=1, shuffle=False, 
                               num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset , batch_size=1, shuffle=False, 
                               num_workers=0, drop_last=True)
test_loader = DataLoader(test_dataset , batch_size=1, shuffle=False, 
                               num_workers=0, drop_last=True)

for idx, img in enumerate(train_loader):
    #do your training now
    print(idx)

[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667
 0.16666667 0.16666667]
0
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667
 0.16666667 0.16666667]
1


KeyError: 2

In [100]:
def get_labels(img):
    date = img.split('_')[2]
    date = date.replace('-', '')
    date = int(date[2:])
    # img_name = img.split('_')[1]
    # img_name = img_name.replace('plot', '')
    if not ("plot" in img):
        print("found image not of plot in no_note_background")
        return False
    plot_num = int(img.split('plot')[1].split("_")[0])
    if plot_num > 0:
        # print(date)
        # print(f'getting labels')
        labels = labels_df[(labels_df['SampleDate'] == date) & (labels_df['plot'] == plot_num)].iloc[0]
        group = labels['group']
        labels = labels.drop(labels=['plot', 'SampleDate', 'Y_cropped', 'group'])
    return labels, group

In [ ]:
def get_data_new(path, resized_path):
    """
    :param path: data path
    :param size: width and length of images (tuple)
    :return:  dictionary of np.arrays of images and labels
    """
    img_list = []
    label_list = []
    group_list = []
    print("Getting data from source...")

    if not is_resize:
        dir_list = os.listdir(resized_path)
    else:
        dir_list = os.listdir(path)
    for i, img in enumerate(dir_list):
        if i > total_img:
            break
        if ".img" in img or "2019-12-25" in img:
            continue
        img_list.append(img)
        labels, group = get_labels(img)
        label_list.append(labels)
        group_list.append(group)

    """split into groups"""
    df = pd.DataFrame(
        {'img': img_list,
         'label': label_list,
         'group': group_list
         })
    df['label'] = df['label'] / label_rank
    df_train = pd.DataFrame(
        columns=['img', 'label', 'group'])
    df_val = pd.DataFrame(
        columns=['img', 'label', 'group'])
    df_test = pd.DataFrame(
        columns=['img', 'label', 'group'])

    for phenotype in range(total_phenotypes - 1):
        df_phenotype = df[df['group'] == phenotype]
        train_df_phenotype = df_phenotype.sample(frac = 0.6)
        val_test_df_phenotype = df_phenotype.drop(train_df_phenotype.index)
        test_df_phenotype = val_test_df_phenotype.sample(frac = 0.5)
        val_df_phenotype = val_test_df_phenotype.drop(test_df_phenotype.index)
        df_train = df_train.append(train_df_phenotype)
        df_val = df_val.append(val_df_phenotype)
        df_test = df_test.append(test_df_phenotype)
    print("Getting data done.")

    return df_train, df_val, df_test

3

# test

In [202]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, image_dir, no_background_dir, no_background_imgdir, transform, is_resize, img_list):
        self.main_dir = main_dir
        self.image_dir = image_dir
        self.no_background_dir = no_background_dir
        self.no_background_imgdir = no_background_imgdir
        self.transform = transform
        self.is_resize = is_resize
        self.labels = img_list['label']
        self.total_imgs = img_list['img']    #natsort.natsorted(img_list['img'])
        self.list_IDs = img_list.index

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        ID = self.list_IDs[idx]
        if self.is_resize:
            hdr_loc = os.path.join(self.no_background_dir, self.total_imgs[ID].replace("resized", "no_note"))
            img_loc = os.path.join(self.no_background_imgdir, self.total_imgs[ID].replace("resized.hdr", "no_note.img"))
            image = envi.open(hdr_loc, img_loc)
            resized = resize(image, size)
            """get mean and std from top 2 rows for background"""
            mean = np.median(image[:, 0:1, :], axis=(0, 1))  # median for every band
            black_std = np.std(image[:, 0:1, :], axis=(0, 1))
            final_image = pad_img(resized, size, mean, black_std, img)
        else:
            hdr_loc = os.path.join(self.main_dir, self.total_imgs[ID])
            img_loc = os.path.join(self.image_dir, self.total_imgs[ID].replace(".hdr", ".img"))
            image = envi.open(hdr_loc, img_loc)
            final_image = np.zeros((image.shape[0], image.shape[1], image.shape[2]))
            final_image[:, :, :] = image[:, :, :]
        x=[]
        #set a seed so the same transforms are applied to each channel
        seed = np.random.randint(2147483647)
        for ch in range(final_image.shape[2]):
            random.seed(seed)
            x.append(transform(Image.fromarray(final_image[:, :, ch])))
        #this is the multichannel transformed image (a torch tensor)
        img_tfm = torch.cat(x)
        y = self.labels[ID].values
        # print(y)
        return img_tfm, y

storage_path = "D:/My Drive/StoragePath"
result_path = storage_path + "/ExpResults/tal_exp_results"
resized_path = result_path + "/resized"
resized_img_path = result_path + "/resizes_img"
no_note_path = result_path + "/no_note_background"
no_note_img_path = result_path + "/no_note_background_img"

tfms = transforms.Compose([transforms.RandomHorizontalFlip(),
                   transforms.ToTensor()])

train_dataset = CustomDataSet(resized_path, resized_img_path, no_note_path, no_note_img_path, transform=tfms, is_resize=False, img_list = df_train)
val_dataset = CustomDataSet(resized_path, resized_img_path, no_note_path, no_note_img_path, transform=tfms, is_resize=False, img_list = df_val)
test_dataset = CustomDataSet(resized_path, resized_img_path, no_note_path, no_note_img_path, transform=tfms, is_resize=False, img_list = df_test)
# x, y = random_split(my_dataset, [20, 9])

train_loader = DataLoader(train_dataset , batch_size=1, shuffle=False, 
                               num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset , batch_size=1, shuffle=False, 
                               num_workers=0, drop_last=True)
test_loader = DataLoader(test_dataset , batch_size=1, shuffle=False, 
                               num_workers=0, drop_last=True)

for idx, img in enumerate(train_loader):
    #do your training now
    print(img)

[tensor([[[[-3.0039e-02,  3.5335e-02,  9.4463e-02,  ...,  4.6150e-02,
           -1.8498e-02, -3.0265e-02],
          [ 1.0431e-01,  7.5541e-02,  5.1028e-02,  ...,  4.7723e-02,
            5.7007e-03, -2.6652e-02],
          [-6.2745e-02,  1.8036e-02,  3.8646e-02,  ..., -2.9577e-02,
            2.7292e-02, -6.6163e-03],
          ...,
          [ 4.5305e-02, -2.8770e-02, -7.1273e-02,  ...,  6.2624e-02,
            9.7766e-02, -2.3828e-02],
          [ 8.5962e-03, -6.8862e-02,  4.3901e-02,  ..., -3.5763e-02,
            3.0148e-02,  4.3918e-02],
          [ 3.0612e-02, -3.7550e-02, -9.7928e-03,  ...,  6.8905e-02,
            4.0799e-02,  3.5339e-02]],

         [[ 1.1667e-02,  3.7401e-02,  9.5108e-02,  ...,  6.0428e-02,
            5.0679e-02, -4.4156e-02],
          [ 9.1834e-02, -7.5532e-03,  6.6181e-02,  ...,  4.7314e-02,
            2.6968e-02,  3.3652e-02],
          [ 8.8658e-02,  1.0117e-01,  5.0792e-02,  ...,  2.7272e-02,
            5.0776e-02,  8.0127e-02],
          ...,
    

KeyboardInterrupt: 

In [191]:
y = df['label'][0].values

In [192]:
y

array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667])

In [198]:
train_dataset.img_list

,img,label,group
20,Corn_plot16_2019-12-22_08-13-38_resized.hdr,necrosis 0.166667 B...,0.0
0,Corn_plot100_2019-12-22_09-25-00_resized.hdr,necrosis 0.166667 B...,0.0
26,Corn_plot18_2019-12-22_08-14-58_resized.hdr,necrosis 0.166667 B...,0.0
13,Corn_plot13_2019-12-29_10-55-31_resized.hdr,necrosis 0.166667 B...,0.0
18,Corn_plot15_2019-12-24_08-33-17_resized.hdr,necrosis 0.166667 B...,0.0
3,Corn_plot10_2019-12-22_08-08-30_resized.hdr,necrosis 0.166667 B...,0.0
10,Corn_plot12_2019-12-24_08-30-45_resized.hdr,necrosis 0.166667 B...,0.0
17,Corn_plot15_2019-12-22_08-12-38_resized.hdr,necrosis 0.166667 B...,0.0
12,Corn_plot13_2019-12-22_08-11-20_resized.hdr,necrosis 0.166667 B...,0.0
1,Corn_plot100_2019-12-24_09-34-05_resized.hdr,necrosis 0.166667 B...,0.0


In [174]:
for idx, img in enumerate(train_loader):
    #do your training now
    print(idx)

730
0
730
1
730
2
730
3
730
4
730
5
730
6
730
7
730
8
730
9
730
10
730
11
730
12
730
13
730
14
730
15
730
16
730
17


In [92]:
storage_path = "D:/My Drive/StoragePath"
result_path = storage_path + "/ExpResults/tal_exp_results"
resized_path = result_path + "/resized"
resized_img_path = result_path + "/resizes_img"
is_resize = False
label_rank = 6
total_phenotypes = 8
data_set_path = storage_path + "/Datasets/tal_datesets/corn_data_set"
labels_df = pd.read_csv(f"{result_path}/results/phenotyping_with_groups_sparse.csv")
total_img = 28


In [160]:
def get_data_new(path, resized_path):
    """
    :param path: data path
    :param size: width and length of images (tuple)
    :return:  dictionary of np.arrays of images and labels
    """
    img_list = []
    label_list = []
    group_list = []
    print("Getting data from source...")

    if not is_resize:
        dir_list = os.listdir(resized_path)
    else:
        dir_list = os.listdir(path)
    for i, img in enumerate(dir_list):
        if i > total_img:
            break
        if ".img" in img or "2019-12-25" in img:
            continue
        img_list.append(img)
        labels, group = get_labels(img)
        label_list.append(labels)
        group_list.append(group)

    """split into groups"""
    df = pd.DataFrame(
        {'img': img_list,
         'label': label_list,
         'group': group_list
         })
    df['label'] = df['label'] / label_rank
    df_train = pd.DataFrame(
        columns=['img', 'label', 'group'])
    df_val = pd.DataFrame(
        columns=['img', 'label', 'group'])
    df_test = pd.DataFrame(
        columns=['img', 'label', 'group'])

    for phenotype in range(total_phenotypes - 1):
        df_phenotype = df[df['group'] == phenotype]
        train_df_phenotype = df_phenotype.sample(frac = 0.6)
        val_test_df_phenotype = df_phenotype.drop(train_df_phenotype.index)
        test_df_phenotype = val_test_df_phenotype.sample(frac = 0.5)
        val_df_phenotype = val_test_df_phenotype.drop(test_df_phenotype.index)
        df_train = df_train.append(train_df_phenotype)
        df_val = df_val.append(val_df_phenotype)
        df_test = df_test.append(test_df_phenotype)
    print("Getting data done.")

    return df_train, df_val, df_test

In [151]:
img_list = []
label_list = []
group_list = []
print("Getting data from source...")

if not is_resize:
    dir_list = os.listdir(resized_path)
else:
    dir_list = os.listdir(path)
for i, img in enumerate(dir_list):
    if i > total_img:
        break
    if ".img" in img or "2019-12-25" in img:
        continue
    img_list.append(img)
    labels, group = get_labels(img)
    label_list.append(labels)
    group_list.append(group)

Getting data from source...


In [157]:
    df = pd.DataFrame(
        {'img': img_list,
         'label': label_list,
         'group': group_list
         })
    df['label'] = df['label'] / label_rank
    df_train = pd.DataFrame(
        columns=['img', 'label', 'group'])
    df_val = pd.DataFrame(
        columns=['img', 'label', 'group'])
    df_test = pd.DataFrame(
        columns=['img', 'label', 'group'])

In [158]:
    for phenotype in range(total_phenotypes - 1):
        df_phenotype = df[df['group'] == phenotype]
        train_df_phenotype = df_phenotype.sample(frac = 0.6)
        val_test_df_phenotype = df_phenotype.drop(train_df_phenotype.index)
        test_df_phenotype = val_test_df_phenotype.sample(frac = 0.5)
        val_df_phenotype = val_test_df_phenotype.drop(test_df_phenotype.index)
        df_train = df_train.append(train_df_phenotype)
        df_val = df_val.append(val_df_phenotype)
        df_test = df_test.append(test_df_phenotype)
    print("Getting data done.")

Getting data done.


In [159]:
df_val

,img,label,group
3,Corn_plot10_2019-12-22_08-08-30_resized.hdr,necrosis 0.166667 B...,0.0
6,Corn_plot11_2019-12-22_08-09-57_resized.hdr,necrosis 0.166667 B...,0.0
9,Corn_plot12_2019-12-22_08-10-41_resized.hdr,necrosis 0.166667 B...,0.0
18,Corn_plot15_2019-12-24_08-33-17_resized.hdr,necrosis 0.166667 B...,0.0
25,Corn_plot17_2019-12-29_09-25-43_resized.hdr,necrosis 0.333333 B...,1.0
5,Corn_plot10_2019-12-29_09-19-07_resized.hdr,necrosis 0.166667 B...,3.0


In [150]:
df_train.append(train_df_phenotype)

,img,label,group
16,Corn_plot14_2019-12-29_09-22-20_resized.hdr,necrosis 0.166667 B...,6.0


In [144]:
temp1 = temp.sample(frac = 0.6)

In [145]:
val_test_df_phenotype = temp.drop(temp1.index)

In [146]:
len(val_test_df_phenotype)

8

In [119]:
temp1

,img,label,group
21,Corn_plot16_2019-12-24_08-34-02_resized.hdr,necrosis 0.166667 B...,0.0
6,Corn_plot11_2019-12-22_08-09-57_resized.hdr,necrosis 0.166667 B...,0.0
9,Corn_plot12_2019-12-22_08-10-41_resized.hdr,necrosis 0.166667 B...,0.0
0,Corn_plot100_2019-12-22_09-25-00_resized.hdr,necrosis 0.166667 B...,0.0
26,Corn_plot18_2019-12-22_08-14-58_resized.hdr,necrosis 0.166667 B...,0.0
14,Corn_plot14_2019-12-22_08-12-00_resized.hdr,necrosis 0.166667 B...,0.0
17,Corn_plot15_2019-12-22_08-12-38_resized.hdr,necrosis 0.166667 B...,0.0
7,Corn_plot11_2019-12-24_08-30-01_resized.hdr,necrosis 0.166667 B...,0.0
13,Corn_plot13_2019-12-29_10-55-31_resized.hdr,necrosis 0.166667 B...,0.0
18,Corn_plot15_2019-12-24_08-33-17_resized.hdr,necrosis 0.166667 B...,0.0


In [161]:
df_train, df_val, df_test = get_data_new("", resized_path)

Getting data from source...
Getting data done.


In [165]:
df_test

,img,label,group
23,Corn_plot17_2019-12-22_08-14-20_resized.hdr,necrosis 0.166667 B...,0.0
14,Corn_plot14_2019-12-22_08-12-00_resized.hdr,necrosis 0.166667 B...,0.0
27,Corn_plot18_2019-12-24_08-35-33_resized.hdr,necrosis 0.166667 B...,0.0
7,Corn_plot11_2019-12-24_08-30-01_resized.hdr,necrosis 0.166667 B...,0.0
11,Corn_plot12_2019-12-29_09-21-13_resized.hdr,necrosis 0.166667 B...,3.0


In [166]:
len(df_test)

5

In [5]:
! pip install natsort

In [61]:
from PIL import Image

In [64]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, image_dir, transform, is_resize):
        self.main_dir = main_dir
        self.image_dir = image_dir
        self.transform = transform
        self.is_resize = is_resize
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        hdr_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        img_loc = os.path.join(self.image_dir, self.total_imgs[idx].replace(".hdr", ".img"))
        image = envi.open(hdr_loc, img_loc)
        if self.is_resize:
            resized = resize(image, size)
            """get mean and std from top 2 rows for background"""
            mean = np.median(image[:, 0:1, :], axis=(0, 1))  # median for every band
            black_std = np.std(image[:, 0:1, :], axis=(0, 1))
            final_image = pad_img(resized, size, mean, black_std, img)
        else:
            final_image = np.zeros((image.shape[0], image.shape[1], image.shape[2]))
            final_image[:, :, :] = image[:, :, :]
        x=[]
        #set a seed so the same transforms are applied to each channel
        seed = np.random.randint(2147483647)
        print(final_image.shape[2])
        for ch in range(final_image.shape[2]):
            random.seed(seed)
            x.append(transform(Image.fromarray(final_image[:, :, ch])))
        #this is the multichannel transformed image (a torch tensor)
        img_tfm = torch.cat(x)
        return img_tfm

storage_path = "D:/My Drive/StoragePath"
result_path = storage_path + "/ExpResults/tal_exp_results"
resized_path = result_path + "/resized"
resized_img_path = result_path + "/resizes_img"
no_note_path = result_path + "/no_note_background"
no_note_img_path = result_path + "/no_note_background_img"

tfms = transforms.Compose([transforms.RandomHorizontalFlip(),
                   transforms.ToTensor()])

my_dataset = CustomDataSet(no_note_path, no_note_img_path, transform=tfms)
x, y = random_split(my_dataset, [20, 9])

train_loader = DataLoader(my_dataset , batch_size=1, shuffle=False, 
                               num_workers=0, drop_last=True)

# for idx, img in enumerate(train_loader):
#     #do your training now
#     print(img)

In [ ]:
def get_labels(img):
    date = img.split('_')[2]
    date = date.replace('-', '')
    date = int(date[2:])
    # img_name = img.split('_')[1]
    # img_name = img_name.replace('plot', '')
    if not ("plot" in img):
        print("found image not of plot in no_note_background")
        return False
    plot_num = int(img.split('plot')[1].split("_")[0])
    if plot_num > 0:
        # print(date)
        # print(f'getting labels')
        labels = labels_df[(labels_df['SampleDate'] == date) & (labels_df['plot'] == plot_num)].iloc[0]
        labels = labels.drop(labels=['plot', 'SampleDate', 'Y_cropped'])
    return labels

In [57]:
import random

In [65]:
x, y = random_split(my_dataset, [20, 9])

In [39]:
x

In [37]:
len(my_dataset.total_imgs)

29

In [50]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [ ]:
tfms = transforms.Compose([transforms.RandomHorizontalFlip(),
                           transforms.RandomVerticalFlip(),
                           transforms.ToTensor()])

x=[]
#set a seed so the same transforms are applied to each channel
seed = np.random.randint(2147483647)
for ch in img:
    random.seed(seed)
    x.append(tfms(Image.fromarray(ch)))

#this is the multichannel transformed image (a torch tensor)
img_tfm = torch.cat(x)

In [ ]:
#use fastai's transforms
tfms = get_transforms(flip_vert=True,max_warp=None,max_rotate=45,max_lighting=None,max_zoom=1.1)

#img is a torch tensor (how many channels you need)
fastai_img = fastai.vision.Image(img)
            
#apply the transforms to the image
for tfm in tfms:
    fastai_img = fastai_img.apply_tfms(tfm,size=224)

#this is the multichannel transformed image (a torch tensor)
img_tfm = fastai_img.data

In [55]:
from fastai.basics import *

ModuleNotFoundError: No module named 'fastai'

In [ ]:
transforms.RandomHorizontalFlip(),
transforms.ToTensor(),

In [52]:
!pip install fastai

  Could not find a version that satisfies the requirement torchvision<0.9,>=0.8 (from fastai) (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.3.0, 0.4.1, 0.5.0)
No matching distribution found for torchvision<0.9,>=0.8 (from fastai)



  Using cached https://files.pythonhosted.org/packages/f3/09/36d4d472d0c953f5d931f4c415f715a909793e5d222af205f3e65c034da3/fastai-2.2.5-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/27/d432a1a51a92343c4e9b172bab94a03cb9886fcb2f49c5e4933ed6e34831/Pillow-8.1.0-cp37-cp37m-win_amd64.whl


In [53]:
print(torch.__version__)

1.1.0


In [32]:
x.indices

[4, 7, 0]

In [34]:
len(train_loader)

14

In [ ]:
for idx, img in enumerate(train_loader):
    #do your training now
    print(idx)

In [23]:
storage_path = "D:/My Drive/StoragePath"
result_path = storage_path + "/ExpResults/tal_exp_results"
resized_path = result_path + "/resized"
resized_img_path = result_path + "/resizes_img"


In [22]:
for idx, img in enumerate(train_loader):
    #do your training now
    print(idx)

EnviDataFileNotFoundError: Unable to determine the ENVI data file name for the given header file. You can specify the data file by passing its name as the optional `image` argument to envi.open.

In [21]:
my_dataset = CustomDataSet(resized_path, transform=None)
train_loader = DataLoader(my_dataset , batch_size=2, shuffle=False, 
                               num_workers=0, drop_last=True)